In [119]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [120]:
#On a les données constructeurs pour un RF

results = pd.read_csv(r"data/results.csv", sep=',')
races = pd.read_csv(r"data/races.csv", sep=',')
circuits = pd.read_csv(r"data/circuits.csv", sep=',')
constructors = pd.read_csv(r"data/constructor_standings.csv", sep=',')

df1 = pd.merge(results[['driverId', 'constructorId', 'raceId', 'positionOrder']], races[['raceId', 'year', 'circuitId']], on = 'raceId')
df2 = pd.merge(df1, constructors[['raceId', 'constructorId', 'position', 'points']], on = ['raceId', 'constructorId'])
df3 = pd.merge(df2, circuits[['circuitId','country']], on = 'circuitId')
df3['winner'] = df3['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df4 = df3.drop(['circuitId', 'constructorId', 'positionOrder'], axis = 1)#.set_index(['raceId', 'driverId'])

label_encoder = LabelEncoder()
df4.loc[:, 'country'] = label_encoder.fit_transform(df3['country'])
df4.loc[:,"position**2"] = df4["position"] ** 2
df4.loc[:,"points**2"] = df4["points"] ** 2
df4 = df4.rename(columns={"points" : 'pointsequipe'})

In [121]:
#On a les données sur les pilotes

drivers = pd.read_csv(r"data\driver_standings.csv", sep=",") #on peut prendre le raceID, driverID, Points, Position, wins
driversINFO = pd.read_csv(r"data\drivers.csv",sep=",") #on prend la date de naisse dob[:4]
races = pd.read_csv(r"data\races.csv",sep=",") #raceID, year, circuitID, date, time
results = pd.read_csv(r"data\results.csv", sep=",") #driverID, raceID, positionORder, grid

results = results[['raceId','driverId','grid','positionOrder']] 
driversINFO = driversINFO[['driverId','dob']]
drivers = drivers[['raceId','driverId','points','wins']]
races = races[['raceId','year','circuitId','date','time']]

#transformation pour avoir la date de naissance des conducteurs
driversINFO.loc[:,'dob'] = pd.to_datetime(driversINFO['dob']).dt.year

#transformation de la date des courses pour avoir le mois 
races.loc[:,'date'] = pd.to_datetime(races['date']).dt.month
races = races.rename(columns={'date' : 'month'})

#retirer une erreur dans les dates et garder uniquement l'heure de la course
races["time"] = races['time'].replace({'\\N' : '12:00:00'})
races['hour'] = pd.to_datetime(races['time']).dt.hour
races = races.drop(columns = ["time"])

# on joint resultat et races sur la base de raceID
df12 = pd.merge(results[['raceId','driverId','grid','positionOrder']], races[['raceId','year','circuitId','month','hour']], on = 'raceId')

# on joint le précédent avec drivers avec drivers info sur driverID
df22 = pd.merge(drivers, driversINFO, on = 'driverId')

df32 = pd.merge(df12,df22, on = ['raceId', 'driverId'])
df32

#on utilise positionOrder pour faire le winner
df32['winner'] = df32['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df32["AgeAtRace"] = df32['year'] - df32['dob']
df42 = df32.drop(['positionOrder','circuitId', 'dob'], axis = 1)#.set_index(['raceId', 'driverId'])

#on retire qui a gagné la course à chaque course
df42["wins"] = df42["wins"] - df42["winner"]

C:\Users\luang\AppData\Local\Temp\ipykernel_12964\3659790136.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  races['hour'] = pd.to_datetime(races['time']).dt.hour


In [122]:
print(df4.columns,"\n")
print(df42.columns)

Index(['driverId', 'raceId', 'year', 'position', 'pointsequipe', 'country',
       'winner', 'position**2', 'points**2'],
      dtype='object') 

Index(['raceId', 'driverId', 'grid', 'year', 'month', 'hour', 'points', 'wins',
       'winner', 'AgeAtRace'],
      dtype='object')


In [123]:
print(df4.head())
print("\n")
print(df42.head())

   driverId  raceId  year  position  pointsequipe country  winner  \
0         1      18  2008         1          14.0       1       1   
1         2      18  2008         3           8.0       1       0   
2         3      18  2008         2           9.0       1       0   
3         4      18  2008         4           5.0       1       0   
4         5      18  2008         1          14.0       1       0   

   position**2  points**2  
0            1      196.0  
1            9       64.0  
2            4       81.0  
3           16       25.0  
4            1      196.0  


   raceId  driverId  grid  year month  hour  points  wins  winner AgeAtRace
0      18         1     1  2008     3     4    10.0     0       1        23
1      18         2     5  2008     3     4     8.0     0       0        31
2      18         3     7  2008     3     4     6.0     0       0        23
3      18         4    11  2008     3     4     5.0     0       0        27
4      18         5     3  2008    

In [124]:
DF = pd.merge(df4[['driverId', 'raceId', 'year', 'position**2', 'points**2']],
              df42[['raceId', 'driverId', 'grid', 'month', 'hour', 'points', 'wins','winner', 'AgeAtRace']],
              on = ['driverId', 'raceId'])

In [125]:
DF 
DFtrain = DF[DF['year'] <= 2020]
DFtest = DF[DF['year'] == 2021]

DFtrain1 = DFtrain[['driverId', 'raceId', 'year', 'position**2', 'points**2','winner']].set_index(['raceId','driverId'])
DFtest1 = DFtest[['driverId', 'raceId', 'year', 'position**2', 'points**2','winner']].set_index(['raceId','driverId'])

DFtrain2 = DFtrain[['driverId', 'raceId', 'year','grid', 'month', 'hour', 'points', 'wins', 'AgeAtRace','winner']].set_index(['raceId','driverId'])
DFtest2 = DFtest[['driverId', 'raceId', 'year', 'grid', 'month', 'hour', 'points', 'wins', 'AgeAtRace','winner']].set_index(['raceId','driverId'])

DFtrain3 = DFtrain[['driverId', 'raceId', 'year','grid', 'month', 'points**2', 'wins', 'AgeAtRace','winner']].set_index(['raceId','driverId'])
DFtest3 = DFtest[['driverId', 'raceId', 'year', 'grid', 'month', 'points**2', 'wins', 'AgeAtRace','winner']].set_index(['raceId','driverId'])


In [126]:
#on importe la fonction qui permet de transformer les probabilités en 1er et podium
def fromPROBA_to_WinnerAndPodium(X_dateSet, Y_dataSet, Model):
    y_pred = Y_dataSet
    probabilite = Model.predict_proba(X_dateSet)
    probabilite = pd.DataFrame(probabilite)
    probabilite = probabilite.drop(columns=(0))
    probabilite = np.array(probabilite)
    y_pred.loc[:,'PREpositionOrder'] = probabilite
    
    Taille = y_pred.index
    raceID = Taille.get_level_values('raceId').unique()
    PodiumOr0 = []
    Top1 = []
    for RACE in raceID:
        driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
        array = np.array(y_pred["PREpositionOrder"][RACE])
        ranks = array.argsort()
        nb = len(driverID)
        ranks = nb - ranks.argsort()
        count = 0
        for DRIVER in driverID:
            rang = ranks[count]
            if rang <= 3:
                PodiumOr0.append(ranks[count])
                if rang == 1:
                    Top1.append(1)
                else:
                    Top1.append(0)
            else:
                PodiumOr0.append(0)
                Top1.append(0)
            count += 1 
        count = 0

    y_pred.loc[:,'winner'] = Top1
    y_pred.loc[:,'positionOrder'] = PodiumOr0
    fin = y_pred
    return fin

In [235]:
XDFtrain1 = DFtrain1.drop(columns="winner")
YDFtrain1 = DFtrain1[["winner"]]
XDFtest1 = DFtest1.drop(columns="winner")
YDFtest1 = DFtest1[["winner"]]

XDFtrain2 = DFtrain2.drop(columns="winner")
YDFtrain2 = DFtrain2[["winner"]]
XDFtest2 = DFtest2.drop(columns="winner")
YDFtest2 = DFtest2[["winner"]]


XDFtrain3 = DFtrain3.drop(columns="winner")
YDFtrain3 = DFtrain3[["winner"]]
XDFtest3 = DFtest3.drop(columns="winner")
YDFtest3 = DFtest3[["winner"]]

In [129]:
#le mmodèle RF sur des données constructeurs
Forest1 = RandomForestClassifier(random_state=14)
RF_GRID_1 = {
    'n_estimators' : [70],
    'max_depth' : [10],
    'max_leaf_nodes' : [100]
}

Forest1 = GridSearchCV(Forest1,RF_GRID_1,scoring='recall')
Forest1.fit(XDFtrain1,YDFtrain1.values.ravel())
Forest1.best_params_


{'max_depth': 10, 'max_leaf_nodes': 100, 'n_estimators': 70}

In [130]:
BestParametre = Forest1.best_params_
Forest1 = RandomForestClassifier(**BestParametre, random_state=14)
Forest1.fit(XDFtrain1,YDFtrain1.values.ravel())

RandomForestClassifier(max_depth=10, max_leaf_nodes=100, n_estimators=70,
                       random_state=14)

In [131]:
Forest1.score(XDFtrain1,YDFtrain1)

0.9578735330180417

In [151]:
y_pred_RF1 = fromPROBA_to_WinnerAndPodium(XDFtest1, YDFtest1, Forest1) # permet d'avoir 64.31%
#y_pred_RF1.to_csv("testRF.csv")

In [133]:
Logistic1 = LogisticRegression(random_state=14)
Logis_GRID_1 = {
    'penalty' : ['l2'],
    'solver' : ['sag'],
    'max_iter' : [9000]
}

Logistic1 = GridSearchCV(Logistic1,Logis_GRID_1,scoring='recall')
Logistic1.fit(XDFtrain1,YDFtrain1.values.ravel())
Logistic1.best_params_

c:\Users\luang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'max_iter': 9000, 'penalty': 'l2', 'solver': 'sag'}

In [134]:
Logistic1 = LogisticRegression(**Logistic1.best_params_, random_state=14)
Logistic1.fit(XDFtrain1,YDFtrain1.values.ravel())

LogisticRegression(max_iter=9000, random_state=14, solver='sag')

In [156]:
Y_pred_Lo1 = fromPROBA_to_WinnerAndPodium(XDFtest1,YDFtest1, Logistic1) # 64.43
#Y_pred_Lo1.to_csv("testLO.csv")

In [136]:
# tester un modèle Gradient Boosting 
GradBoost1 = GradientBoostingClassifier(random_state=14)
GradBoost_GRID_1 = {
    'loss' : ['log_loss'],
    'learning_rate' : [0.1],
    'n_estimators' : [100]
}

GradBoost1 = GridSearchCV(GradBoost1,GradBoost_GRID_1,scoring='recall')
GradBoost1.fit(XDFtrain1,YDFtrain1.values.ravel())
GradBoost1.best_params_

{'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 100}

In [137]:
GradBoost1 = GradientBoostingClassifier(**GradBoost1.best_params_, random_state=14)
GradBoost1.fit(XDFtrain1,YDFtrain1.values.ravel())


In [149]:
Y_pred_GradBoost1 = fromPROBA_to_WinnerAndPodium(XDFtest1,YDFtest1, GradBoost1) # 63.97
#Y_pred_GradBoost1.to_csv("testGB.csv")

In [146]:
Y_pred_GradBoost1#, y_pred_RF1,Y_pred_Lo1

winner  PREpositionOrder  positionOrder
raceId driverId                                         
1052   1              1          0.388073              1
       830            0          0.063103              3
       822            0          0.388073              2
       846            0          0.024717              0
       815            0          0.063103              0
...                 ...               ...            ...
1073   849            0          0.004095              0
       841            0          0.003540              0
       847            0          0.004095              0
       8              0          0.003540              0
       853            0          0.000440              0

[440 rows x 3 columns]

In [152]:
y_pred_RF1

winner  PREpositionOrder  positionOrder
raceId driverId                                         
1052   1              1          0.421955              1
       830            0          0.046647              3
       822            0          0.421955              2
       846            0          0.024026              0
       815            0          0.046647              0
...                 ...               ...            ...
1073   849            0          0.009423              0
       841            0          0.007391              0
       847            0          0.009423              0
       8              0          0.007391              0
       853            0          0.004124              0

[440 rows x 3 columns]

In [157]:
Y_pred_Lo1

winner  PREpositionOrder  positionOrder
raceId driverId                                         
1052   1              1          0.044414              1
       830            0          0.043589              3
       822            0          0.044414              2
       846            0          0.042603              0
       815            0          0.043589              0
...                 ...               ...            ...
1073   849            0          0.034537              0
       841            0          0.032245              0
       847            0          0.034537              0
       8              0          0.032245              0
       853            0          0.029914              0

[440 rows x 3 columns]

In [255]:
#on va tester de faire un algo qui prend en entrée toutes les predictions d'avant

PredctionGB1 = pd.read_csv(r'MetaModel3\testGB.csv',sep=',').rename(columns={"winner" :"winnerGb1",	"PREpositionOrder":	"PREpositionOrderGb1","positionOrder":"positionOrderGb1"})
PredctionLo1 = pd.read_csv(r'MetaModel3\testLO.csv',sep=',').rename(columns={"winner" :"winnerLo1",	"PREpositionOrder":	"PREpositionOrderLo1","positionOrder":"positionOrderLo1"})
PredctionRF1 = pd.read_csv(r'MetaModel3\testRF.csv',sep=',').rename(columns={"winner" :"winnerRF1",	"PREpositionOrder":	"PREpositionOrderRF1","positionOrder":"positionOrderRF1"})



In [259]:
METAModel3_v1 = pd.merge(PredctionGB1,PredctionLo1, on=['raceId','driverId'])
METAModel3_v1 = pd.merge(METAModel3_v1,PredctionRF1, on=['raceId','driverId'])

In [260]:
METAModel3_v1 = METAModel3_v1.set_index(['raceId','driverId'])
METAModel3_v1

winnerGb1  PREpositionOrderGb1  positionOrderGb1  winnerLo1  \
raceId driverId                                                                
1052   1                 1             0.388073                 1          1   
       830               0             0.063103                 3          0   
       822               0             0.388073                 2          0   
       846               0             0.024717                 0          0   
       815               0             0.063103                 0          0   
...                    ...                  ...               ...        ...   
1073   849               0             0.004095                 0          0   
       841               0             0.003540                 0          0   
       847               0             0.004095                 0          0   
       8                 0             0.003540                 0          0   
       853               0             0.000440                 0          0   

                 PREpositionOrderLo1  positionOrderLo1  winnerRF1  \
raceId driverId                                                     
1052   1                    0.044414                 1          1   
       830                  0.043589                 3          0   
       822                  0.044414                 2          0   
       846                  0.042603                 0          0   
       815                  0.043589                 0          0   
...                              ...               ...        ...   
1073   849                  0.034537                 0          0   
       841                  0.032245                 0          0   
       847                  0.034537                 0          0   
       8                    0.032245                 0          0   
       853                  0.029914                 0          0   

                 PREpositionOrderRF1  positionOrderRF1  
raceId driverId                                         
1052   1                    0.421955                 1  
       830                  0.046647                 3  
       822                  0.421955                 2  
       846                  0.024026                 0  
       815                  0.046647                 0  
...                              ...               ...  
1073   849                  0.009423                 0  
       841                  0.007391                 0  
       847                  0.009423                 0  
       8                    0.007391                 0  
       853                  0.004124                 0  

[440 rows x 9 columns]

In [261]:
def fromIndexToColumn(arrayPandas):
    A = pd.DataFrame(arrayPandas, index = arrayPandas.index)
    A = A.reset_index()
    return A

In [271]:
#train le metamodel3_v1
y_pred_metamodel3_RF1 = fromPROBA_to_WinnerAndPodium(XDFtrain1, YDFtrain1, Forest1)
y_pred_metamodel3_RF1 = y_pred_metamodel3_RF1.rename(columns={"winner" :"winnerRF1",	"PREpositionOrder":	"PREpositionOrderRF1","positionOrder":"positionOrderRF1"})
y_pred_metamodel3_RF1 = fromIndexToColumn(y_pred_metamodel3_RF1)

y_pred_metamodel3_Lo1 = fromPROBA_to_WinnerAndPodium(XDFtrain1, YDFtrain1, Logistic1)
y_pred_metamodel3_Lo1 = y_pred_metamodel3_Lo1.rename(columns={"winner" :"winnerLo1",	"PREpositionOrder":	"PREpositionOrderLo1","positionOrder":"positionOrderLo1"})
y_pred_metamodel3_Lo1 = fromIndexToColumn(y_pred_metamodel3_Lo1)

y_pred_metamodel3_Gb1 = fromPROBA_to_WinnerAndPodium(XDFtrain1, YDFtrain1, GradBoost1)
y_pred_metamodel3_Gb1 = y_pred_metamodel3_Gb1.rename(columns={"winner" :"winnerGb1",	"PREpositionOrder":	"PREpositionOrderGb1","positionOrder":"positionOrderGb1"})
y_pred_metamodel3_Gb1 = fromIndexToColumn(y_pred_metamodel3_Gb1)


In [283]:
Xtrain_pred_metamodel3_RF = pd.merge(y_pred_metamodel3_Gb1,y_pred_metamodel3_Lo1, on=['raceId','driverId'])
Xtrain_pred_metamodel3_RF = pd.merge(Xtrain_pred_metamodel3_RF,y_pred_metamodel3_RF1, on=['raceId','driverId'])
Xtrain_pred_metamodel3_RF = Xtrain_pred_metamodel3_RF#.set_index(['raceId','driverId'])

In [284]:
ALL_TrainMetaModel3 = pd.merge(Xtrain_pred_metamodel3_RF, YDFtrain1[['winner']], on=['raceId','driverId'])
ALL_TrainMetaModel3 = ALL_TrainMetaModel3.set_index(['raceId','driverId'])
ALL_TrainMetaModel3_X = ALL_TrainMetaModel3.drop(columns=('winner'))
ALL_TrainMetaModel3_Y = ALL_TrainMetaModel3['winner']

In [285]:
ALL_TrainMetaModel3

winnerGb1  PREpositionOrderGb1  positionOrderGb1  winnerLo1  \
raceId driverId                                                                
18     1                 0             0.438136                 2          0   
       2                 0             0.009230                 0          0   
       3                 0             0.076377                 0          0   
       4                 0             0.001112                 0          0   
       5                 1             0.438136                 1          1   
...                    ...                  ...               ...        ...   
1047   841               0             0.001000                 0          0   
       849               0             0.000440                 0          0   
       825               0             0.000440                 0          0   
       850               0             0.000440                 0          0   
       815               0             0.012801                 0          0   

                 PREpositionOrderLo1  positionOrderLo1  winnerRF1  \
raceId driverId                                                     
18     1                    0.044720                 2          0   
       2                    0.043326                 0          0   
       3                    0.044169                 0          0   
       4                    0.042167                 0          0   
       5                    0.044720                 1          1   
...                              ...               ...        ...   
1047   841                  0.034457                 0          0   
       849                  0.029958                 0          0   
       825                  0.032250                 0          0   
       850                  0.032250                 0          0   
       815                  0.056323                 0          0   

                 PREpositionOrderRF1  positionOrderRF1  winner  
raceId driverId                                                 
18     1                    0.380221                 2       0  
       2                    0.009814                 0       0  
       3                    0.020405                 0       0  
       4                    0.004830                 0       0  
       5                    0.380221                 1       1  
...                              ...               ...     ...  
1047   841                  0.004138                 0       0  
       849                  0.004124                 0       0  
       825                  0.004124                 0       0  
       850                  0.004124                 0       0  
       815                  0.027931                 0       0  

[24866 rows x 10 columns]

In [297]:
randomForestMetaModel = RandomForestClassifier(random_state=14)
MetaModelRF_GRID = {
    'n_estimators' : [5],
    'max_depth' : [2],
    'max_leaf_nodes' : [5]
}

randomForestMetaModel = GridSearchCV(randomForestMetaModel,MetaModelRF_GRID,scoring='recall')
randomForestMetaModel.fit(ALL_TrainMetaModel3_X,ALL_TrainMetaModel3_Y.values.ravel())
randomForestMetaModel.best_params_


{'max_depth': 2, 'max_leaf_nodes': 5, 'n_estimators': 5}

In [298]:
randomForestMetaModel.score(ALL_TrainMetaModel3_X,ALL_TrainMetaModel3_Y.values.ravel())

1.0

In [299]:
ALL_TrainMetaModel3_X

winnerGb1  PREpositionOrderGb1  positionOrderGb1  winnerLo1  \
raceId driverId                                                                
18     1                 0             0.438136                 2          0   
       2                 0             0.009230                 0          0   
       3                 0             0.076377                 0          0   
       4                 0             0.001112                 0          0   
       5                 1             0.438136                 1          1   
...                    ...                  ...               ...        ...   
1047   841               0             0.001000                 0          0   
       849               0             0.000440                 0          0   
       825               0             0.000440                 0          0   
       850               0             0.000440                 0          0   
       815               0             0.012801                 0          0   

                 PREpositionOrderLo1  positionOrderLo1  winnerRF1  \
raceId driverId                                                     
18     1                    0.044720                 2          0   
       2                    0.043326                 0          0   
       3                    0.044169                 0          0   
       4                    0.042167                 0          0   
       5                    0.044720                 1          1   
...                              ...               ...        ...   
1047   841                  0.034457                 0          0   
       849                  0.029958                 0          0   
       825                  0.032250                 0          0   
       850                  0.032250                 0          0   
       815                  0.056323                 0          0   

                 PREpositionOrderRF1  positionOrderRF1  
raceId driverId                                         
18     1                    0.380221                 2  
       2                    0.009814                 0  
       3                    0.020405                 0  
       4                    0.004830                 0  
       5                    0.380221                 1  
...                              ...               ...  
1047   841                  0.004138                 0  
       849                  0.004124                 0  
       825                  0.004124                 0  
       850                  0.004124                 0  
       815                  0.027931                 0  

[24866 rows x 9 columns]

In [289]:
METAModel3_v1

winnerGb1  PREpositionOrderGb1  positionOrderGb1  winnerLo1  \
raceId driverId                                                                
1052   1                 1             0.388073                 1          1   
       830               0             0.063103                 3          0   
       822               0             0.388073                 2          0   
       846               0             0.024717                 0          0   
       815               0             0.063103                 0          0   
...                    ...                  ...               ...        ...   
1073   849               0             0.004095                 0          0   
       841               0             0.003540                 0          0   
       847               0             0.004095                 0          0   
       8                 0             0.003540                 0          0   
       853               0             0.000440                 0          0   

                 PREpositionOrderLo1  positionOrderLo1  winnerRF1  \
raceId driverId                                                     
1052   1                    0.044414                 1          1   
       830                  0.043589                 3          0   
       822                  0.044414                 2          0   
       846                  0.042603                 0          0   
       815                  0.043589                 0          0   
...                              ...               ...        ...   
1073   849                  0.034537                 0          0   
       841                  0.032245                 0          0   
       847                  0.034537                 0          0   
       8                    0.032245                 0          0   
       853                  0.029914                 0          0   

                 PREpositionOrderRF1  positionOrderRF1  
raceId driverId                                         
1052   1                    0.421955                 1  
       830                  0.046647                 3  
       822                  0.421955                 2  
       846                  0.024026                 0  
       815                  0.046647                 0  
...                              ...               ...  
1073   849                  0.009423                 0  
       841                  0.007391                 0  
       847                  0.009423                 0  
       8                    0.007391                 0  
       853                  0.004124                 0  

[440 rows x 9 columns]

In [ ]:
YDFtrain1

In [300]:
pred_metaModel_V1 = fromPROBA_to_WinnerAndPodium(METAModel3_v1,YDFtest1,randomForestMetaModel)#.drop(columns=('year','position**2',"PREpositionOrder", 'points**2'))
pred_metaModel_V1.to_csv("MetaModel3/metamodelonlyConstructer.csv") #64.20%

In [306]:
GradientMetaModel = GradientBoostingClassifier(random_state=14)
MetaModelGB_GRID = {
    'loss' : ['log_loss'],
    'learning_rate' : [0.1],
    'n_estimators' : [4]
}

GradientMetaModel = GridSearchCV(GradientMetaModel,MetaModelGB_GRID,scoring='recall')
GradientMetaModel.fit(ALL_TrainMetaModel3_X,ALL_TrainMetaModel3_Y.values.ravel())
GradientMetaModel.best_params_
print(GradientMetaModel.score(ALL_TrainMetaModel3_X,ALL_TrainMetaModel3_Y.values.ravel()))

1.0


In [307]:
pred_metaModel_V2 = fromPROBA_to_WinnerAndPodium(METAModel3_v1,YDFtest1,GradientMetaModel)#.drop(columns=('year','position**2',"PREpositionOrder", 'points**2'))
pred_metaModel_V2.to_csv("MetaModel3/metamodelonlyConstructer_V2.csv") #62.15

DFtrain2

In [315]:
GradBoost = GradientBoostingClassifier(random_state=48)
GradBoost2_GRID = {
    'loss' : ['log_loss'],
    'learning_rate' : [0.4],
    'n_estimators' : [15]
}
GradBoost = GridSearchCV(GradBoost, GradBoost2_GRID, scoring='recall')
GradBoost.fit(XDFtrain2,YDFtrain2.values.ravel())
GradBoost.best_params_

{'learning_rate': 0.4, 'loss': 'log_loss', 'n_estimators': 15}

In [316]:
GradBoost = GradientBoostingClassifier(**GradBoost.best_params_, random_state=48)
GradBoost.fit(XDFtrain2,YDFtrain2.values.ravel())
print(GradBoost.score(XDFtrain2,YDFtrain2.values.ravel()))

0.9644421089507794


In [317]:
pred_GradBoostDriver_V1 = fromPROBA_to_WinnerAndPodium(XDFtest2,YDFtest2, GradBoost)
pred_GradBoostDriver_V1.to_csv('MetaModel3/GradBoostV1.csv') #67.5

C:\Users\luang\AppData\Local\Temp\ipykernel_12964\4057042677.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'PREpositionOrder'] = probabilite
C:\Users\luang\AppData\Local\Temp\ipykernel_12964\4057042677.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'positionOrder'] = PodiumOr0


In [319]:
ModelLogistic_V1 = LogisticRegression(random_state=14)
ModelLogistic_V1_Grid = {
    'penalty': ["l1"],
    'max_iter': [3800],
    'solver' :["liblinear"]
}

ModelLogistic_V1 = GridSearchCV(ModelLogistic_V1, ModelLogistic_V1_Grid, scoring='recall')
ModelLogistic_V1.fit(XDFtrain2,YDFtrain2.values.ravel())
ModelLogistic_V1 = LogisticRegression(**ModelLogistic_V1.best_params_, random_state=14)

ModelLogistic_V1.fit(XDFtrain2,YDFtrain2.values.ravel())
print(ModelLogistic_V1.score(XDFtrain2,YDFtrain2.values.ravel()))

0.9575232089682957


In [320]:
pred_ModelLogistic_V1 = fromPROBA_to_WinnerAndPodium(XDFtest2,YDFtest2, ModelLogistic_V1)
pred_ModelLogistic_V1.to_csv('MetaModel3/ModelLogistic_V1.csv') #69.77